# MCP Connectivity Test

Simple notebook to test MCP server connectivity using the low-level MCP client.  
No LangGraph or LLM required - just direct MCP protocol calls.

Uses `terminate_on_close=False` which is required for AgentCore Gateway.

## 1. Verify Packages

In [ ]:
import importlib.metadata

packages = ["mcp", "httpx", "nest-asyncio"]
for pkg in packages:
    try:
        v = importlib.metadata.version(pkg)
        print(f"{pkg}: {v}")
    except importlib.metadata.PackageNotFoundError:
        print(f"{pkg}: NOT FOUND - run: pip install {pkg}")

## 2. Imports

In [ ]:
import asyncio
from datetime import timedelta
import nest_asyncio

nest_asyncio.apply()

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

print("Imports OK")

## 3. Configuration

Copy from `.mcp-credentials.json`

In [ ]:
GATEWAY_URL = "YOUR_GATEWAY_URL_HERE"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"

if "YOUR_" in GATEWAY_URL or "YOUR_" in ACCESS_TOKEN:
    print("ERROR: Replace GATEWAY_URL and ACCESS_TOKEN")
else:
    print(f"Gateway: {GATEWAY_URL[:60]}...")
    print(f"Token:   {ACCESS_TOKEN[:30]}...")
    print("Config OK")

## 4. List Available Tools

Uses `terminate_on_close=False` for AgentCore Gateway compatibility.

In [ ]:
async def list_tools_async():
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    async with streamablehttp_client(
        GATEWAY_URL,
        headers,
        timeout=timedelta(seconds=120),
        terminate_on_close=False
    ) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            result = await session.list_tools()
            return result.tools

tools = asyncio.get_event_loop().run_until_complete(list_tools_async())

print(f"Found {len(tools)} tools:")
print("-" * 50)
for tool in tools:
    print(f"  {tool.name}")

## 5. Tool Call Helper

Handles Gateway tool name prefixing (e.g., `target___get-schema`).

In [ ]:
async def call_tool_async(tool_name: str, args: dict = None):
    """Call an MCP tool with automatic name resolution."""
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    async with streamablehttp_client(
        GATEWAY_URL,
        headers,
        timeout=timedelta(seconds=120),
        terminate_on_close=False
    ) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()

            # Build tool name map (handle Gateway prefix)
            tools_result = await session.list_tools()
            tool_map = {}
            for t in tools_result.tools:
                full_name = t.name
                base_name = full_name.split("___")[-1] if "___" in full_name else full_name
                tool_map[base_name] = full_name

            # Resolve tool name
            resolved_name = tool_map.get(tool_name, tool_name)
            print(f"Resolved: {tool_name} -> {resolved_name}")

            # Call tool
            result = await session.call_tool(resolved_name, args or {})
            return result

def run_tool(name: str, args: dict = None):
    """Sync wrapper with pretty output."""
    print(f"Tool: {name}")
    if args:
        print(f"Args: {args}")
    print("-" * 50)

    result = asyncio.get_event_loop().run_until_complete(call_tool_async(name, args))

    # Debug: show result structure
    print(f"Result type: {type(result).__name__}")

    if hasattr(result, 'content') and result.content:
        print(f"Content items: {len(result.content)}")
        for i, item in enumerate(result.content):
            item_type = type(item).__name__
            print(f"\n--- Item {i} ({item_type}) ---")
            if hasattr(item, 'text'):
                text = item.text
                print(text[:2000] if len(text) > 2000 else text)
            elif hasattr(item, 'data'):
                print(f"Data: {item.data}")
            else:
                print(f"Raw: {item}")
    elif hasattr(result, 'isError') and result.isError:
        print(f"ERROR: {result}")
    else:
        print(f"Raw result: {result}")

    return result

print("Helper ready")

## 6. Test: Get Schema

In [ ]:
_ = run_tool("get-schema")

## 7. Test: Run Cypher Query

In [ ]:
_ = run_tool("read-cypher", {"query": "MATCH (n) RETURN labels(n) AS label, count(*) AS count"})

## 8. Custom Queries

In [ ]:
# Example:
# _ = run_tool("read-cypher", {"query": "MATCH (n:Person) RETURN n.name LIMIT 5"})